In [13]:
import spacy
import torch
from transformers import CamembertTokenizer, CamembertForTokenClassification
from transformers import pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
import seaborn as sns
import tensorflow as tf

In [19]:
print("Vérification GPU...")
if tf.config.list_physical_devices('GPU'):
    print("GPU détecté")
else:
    print("Aucun GPU détecté, l'entraînement se fera sur le CPU")

Vérification GPU...
Aucun GPU détecté, l'entraînement se fera sur le CPU


In [47]:
def load_ner_dataset_file():
    ner_data = pd.read_csv("../../../../students_bootstrap/corpus/ner_dataset.csv", encoding='latin1', delimiter=",")
    print("Loaded ner_dataset file\n")
    return ner_data
    
def load_bottins_file():
    bottins_data = pd.read_csv("../../../../students_bootstrap/bottins.csv", encoding="utf-8", delimiter=",", header=None)
    print("Loaded bottins file \n")
    return bottins_data

ner_data = load_ner_dataset_file()
print("Premières lignes du dataset ner_dataset.csv :\n")
print(ner_data.head())

print("\n######################################################################\n")

bottins_data = load_bottins_file()
print("Premières lignes du dataset bottins.csv :\n")
print(bottins_data.head())

Loaded ner_dataset file
Premières lignes du dataset ner_dataset.csv :
    Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O

######################################################################

Loaded bottins file
Premières lignes du dataset bottins.csv :
                                                   0                1
0  <PER>Dufan et Clémendot</PER>, <ACT>pharmacien...   "Bottin1_1820"
1  <PER>Dufant (Victor)</PER>, <ACT>libraire</ACT...   "Bottin1_1820"
2  <PER>Dufay</PER>, <ACT>essayeur du commerce</A...   "Bottin1_1820"
3  <PER>Dulay</PER>, <ACT>chandronnier</ACT>, <LO...   "Bottin1_1820"
4  <PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...   "Bottin1_1820"
